<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Material adaptado para la materia <strong>Introducción al Procesamiento del Habla y LLMs</strong><br>
Basado en el libro <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> de <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Repositorio original: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
</tr>
</table>

# Intro a LLMs

##Trabajando con Datos de Texto

## Configuración para Google Colab

Si estás ejecutando este cuaderno en Google Colab, primero necesitamos descargar el archivo de texto:

In [1]:
# Descargar el texto de Borges si no existe
import os
import requests
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo en Google Drive (asegúrate de que esta ruta sea correcta)
# Por ejemplo, si el archivo está en la carpeta "Mi Drive"
ruta_archivo_drive = "/content/drive/MyDrive/Clases/000 - 2DO Cuatrimestre/HABLA/009 - Intro a LLMs/PRA/001-texto/001-texto/tlon-uqbar-orbis-tertius.txt"

# Verificar si el archivo existe en Drive antes de intentar leerlo
if not os.path.exists(ruta_archivo_drive):
    print(f"Error: El archivo no se encontró en la ruta especificada: {ruta_archivo_drive}")
    print("Asegúrate de que el archivo 'tlon-uqbar-orbis-tertius.txt' esté en tu Google Drive")
    print("y que la ruta en el código sea correcta.")
else:
    print(f"Archivo encontrado en: {ruta_archivo_drive}")

Mounted at /content/drive
Archivo encontrado en: /content/drive/MyDrive/Clases/000 - 2DO Cuatrimestre/HABLA/009 - Intro a LLMs/PRA/001-texto/001-texto/tlon-uqbar-orbis-tertius.txt


## Instalación de paquetes

Paquetes que se utilizan en este cuaderno:

In [2]:
# Instalar dependencias en Colab
!pip install -q torch tiktoken

In [3]:
from importlib.metadata import version

print("Versión de torch:", version("torch"))
print("Versión de tiktoken:", version("tiktoken"))

Versión de torch: 2.8.0+cu126
Versión de tiktoken: 0.12.0


- Este capítulo cubre la preparación y el muestreo de datos para preparar los datos de entrada para un LLM
- Trabajaremos con el cuento "Tlön, Uqbar, Orbis Tertius" de Jorge Luis Borges

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 Entendiendo los word embeddings

- No hay código en esta sección

- Existen muchas formas de embeddings; en este libro nos enfocamos en embeddings de texto

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- Los LLMs trabajan con embeddings en espacios de alta dimensionalidad (es decir, miles de dimensiones)
- Como no podemos visualizar espacios de tan alta dimensionalidad (los humanos pensamos en 1, 2 o 3 dimensiones), la figura siguiente ilustra un espacio de embeddings de 2 dimensiones

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 Tokenizando texto

- En esta sección, tokenizamos texto, lo que significa dividir el texto en unidades más pequeñas, como palabras individuales y caracteres de puntuación

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- Cargamos el texto con el que queremos trabajar
- ["Tlön, Uqbar, Orbis Tertius" de Jorge Luis Borges](https://ciudadseva.com/texto/tlon-uqbar-orbis-tertius/)

In [3]:
# Utilizamos la ruta del archivo en Google Drive que verificamos anteriormente
ruta_archivo_drive = "/content/drive/MyDrive/Clases/000 - 2DO Cuatrimestre/HABLA/009 - Intro a LLMs/PRA/001-texto/001-texto/tlon-uqbar-orbis-tertius.txt"

with open(ruta_archivo_drive, "r", encoding="utf-8") as archivo:
    texto_crudo = archivo.read()

print("Número total de caracteres:", len(texto_crudo))
print(texto_crudo[:200])

Número total de caracteres: 32516
Debo a la conjunción de un espejo y de una enciclopedia el descubrimiento de Uqbar. El espejo inquietaba el fondo de un corredor en una quinta de la calle Gaona, en Ramos Mejía; la enciclopedia falazm


- El objetivo es tokenizar y embeber este texto para un LLM
- Desarrollemos un tokenizador simple basado en un texto de ejemplo que después podamos aplicar al texto completo
- La siguiente expresión regular dividirá el texto en espacios en blanco

In [4]:
import re

texto = "Hola, mundo. Esto, es una prueba."
resultado = re.split(r'(\s)', texto)

print(resultado)

['Hola,', ' ', 'mundo.', ' ', 'Esto,', ' ', 'es', ' ', 'una', ' ', 'prueba.']


- No solo queremos dividir por espacios en blanco sino también por comas y puntos, así que modifiquemos la expresión regular para hacer eso también

In [5]:
resultado = re.split(r'([,.]|\s)', texto)

print(resultado)

['Hola', ',', '', ' ', 'mundo', '.', '', ' ', 'Esto', ',', '', ' ', 'es', ' ', 'una', ' ', 'prueba', '.', '']


- Como podemos ver, esto crea strings vacías, eliminémoslas

In [6]:
# Eliminamos los espacios en blanco de cada ítem y luego filtramos cualquier string vacía.
resultado = [item for item in resultado if item.strip()]
print(resultado)

['Hola', ',', 'mundo', '.', 'Esto', ',', 'es', 'una', 'prueba', '.']


- Esto se ve bastante bien, pero también manejemos otros tipos de puntuación, como puntos, signos de interrogación, y demás

In [7]:
texto = "Hola, mundo. ¿Es esto-- una prueba?"

resultado = re.split(r'([,.:;?_!"()\']|--|\s)', texto)
resultado = [item.strip() for item in resultado if item.strip()]
print(resultado)

['Hola', ',', 'mundo', '.', '¿Es', 'esto', '--', 'una', 'prueba', '?']


- Esto está bastante bien, y ahora estamos listos para aplicar esta tokenización al texto completo

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
preprocesado = re.split(r'([,.:;?_!"()\']|--|\s)', texto_crudo)
preprocesado = [item.strip() for item in preprocesado if item.strip()]
print(preprocesado[:30])

['Debo', 'a', 'la', 'conjunción', 'de', 'un', 'espejo', 'y', 'de', 'una', 'enciclopedia', 'el', 'descubrimiento', 'de', 'Uqbar', '.', 'El', 'espejo', 'inquietaba', 'el', 'fondo', 'de', 'un', 'corredor', 'en', 'una', 'quinta', 'de', 'la', 'calle']


- Calculemos el número total de tokens

In [9]:
print(len(preprocesado))

6109


## 2.3 Convirtiendo tokens en IDs de tokens

- A continuación, convertimos los tokens de texto en IDs de tokens que podemos procesar mediante capas de embedding más adelante

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- A partir de estos tokens, ahora podemos construir un vocabulario que consiste en todos los tokens únicos

In [10]:
todas_palabras = sorted(set(preprocesado))
tamaño_vocab = len(todas_palabras)

print(tamaño_vocab)

2104


In [11]:
vocabulario = {token: entero for entero, token in enumerate(todas_palabras)}

- A continuación se muestran las primeras 50 entradas en este vocabulario:

In [12]:
for i, item in enumerate(vocabulario.items()):
    print(item)
    if i >= 50:
        break

('(', 0)
(')', 1)
(',', 2)
('-Dios', 3)
('-Jorasán', 4)
('-Silas', 5)
('-a', 6)
('-con', 7)
('-congénitamente-', 8)
('-de', 9)
('-el', 10)
('-interrogaron-', 11)
('-la', 12)
('-los', 13)
('-meses', 14)
('-más', 15)
('-ni', 16)
('-o', 17)
('-que', 18)
('-siquiera', 19)
('-tal', 20)
('-traduzco', 21)
('-y', 22)
('.', 23)
('1', 24)
('10', 25)
('1001', 26)
('1641', 27)
('1824', 28)
('1828', 29)
('1874-figura', 30)
('1902', 31)
('1914', 32)
('1917', 33)
('1937', 34)
('1940', 35)
('1941', 36)
('1942', 37)
('1944', 38)
('1947', 39)
('2', 40)
('4', 41)
('5', 42)
('917', 43)
('918', 44)
('920', 45)
('921', 46)
(':', 47)
(';', 48)
('>', 49)
('?', 50)


- A continuación, ilustramos la tokenización de un texto de ejemplo corto usando un vocabulario pequeño:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- Juntando todo ahora en una clase tokenizadora

In [13]:
class TokenizadorSimpleV1:
    def __init__(self, vocabulario):
        self.str_a_int = vocabulario
        self.int_a_str = {i: s for s, i in vocabulario.items()}

    def codificar(self, texto):
        preprocesado = re.split(r'([,.:;?_!"()\']|--|\s)', texto)

        preprocesado = [
            item.strip() for item in preprocesado if item.strip()
        ]
        ids = [self.str_a_int[s] for s in preprocesado]
        return ids

    def decodificar(self, ids):
        texto = " ".join([self.int_a_str[i] for i in ids])
        # Reemplazamos espacios antes de las puntuaciones especificadas
        texto = re.sub(r'\s+([,.?!"()\'])', r'\1', texto)
        return texto

- La función `codificar` convierte texto en IDs de tokens
- La función `decodificar` convierte IDs de tokens de vuelta a texto

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- Podemos usar el tokenizador para codificar (es decir, tokenizar) textos en enteros
- Estos enteros pueden entonces ser embebidos (más adelante) como entrada para el LLM

In [14]:
tokenizador = TokenizadorSimpleV1(vocabulario)

texto = """Debo a la conjunción de un espejo y de una enciclopedia
           el descubrimiento de Uqbar."""
ids = tokenizador.codificar(texto)
print(ids)

[112, 326, 1238, 622, 703, 1992, 919, 2064, 703, 1993, 871, 849, 747, 703, 307, 23]


### Veamos qué texto representan estos IDs:

In [15]:
print("IDs:", ids)
print("\nTexto decodificado:")
print(tokenizador.decodificar(ids))

IDs: [112, 326, 1238, 622, 703, 1992, 919, 2064, 703, 1993, 871, 849, 747, 703, 307, 23]

Texto decodificado:
Debo a la conjunción de un espejo y de una enciclopedia el descubrimiento de Uqbar.


- Podemos decodificar los enteros de vuelta a texto

In [16]:
tokenizador.decodificar(ids)

'Debo a la conjunción de un espejo y de una enciclopedia el descubrimiento de Uqbar.'

In [17]:
tokenizador.decodificar(tokenizador.codificar(texto))

'Debo a la conjunción de un espejo y de una enciclopedia el descubrimiento de Uqbar.'

In [18]:
tokenizador.codificar(texto)

[112,
 326,
 1238,
 622,
 703,
 1992,
 919,
 2064,
 703,
 1993,
 871,
 849,
 747,
 703,
 307,
 23]

## 2.4 Agregando tokens de contexto especiales

- Es útil agregar algunos tokens "especiales" para palabras desconocidas y para denotar el final de un texto

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- Algunos tokenizadores usan tokens especiales para ayudar al LLM con contexto adicional
- Algunos de estos tokens especiales son:
  - `[BOS]` (beginning of sequence) marca el comienzo del texto
  - `[EOS]` (end of sequence) marca dónde termina el texto (esto generalmente se usa para concatenar múltiples textos no relacionados, por ejemplo, dos artículos diferentes de Wikipedia o dos libros diferentes, etc.)
  - `[PAD]` (padding) si entrenamos LLMs con un tamaño de batch mayor que 1 (podemos incluir múltiples textos con diferentes longitudes; con el token de padding rellenamos los textos más cortos a la longitud más larga para que todos los textos tengan la misma longitud)
  - `[UNK]` para representar palabras que no están incluidas en el vocabulario

- Notemos que GPT-2 no necesita ninguno de los tokens mencionados arriba pero solo usa un token `<|endoftext|>` para reducir la complejidad
- El `<|endoftext|>` es análogo al token `[EOS]` mencionado arriba
- GPT también usa el `<|endoftext|>` para padding (ya que típicamente usamos una máscara cuando entrenamos con entradas en batch, no atenderíamos a los tokens de padding de todos modos, así que no importa cuáles sean estos tokens)
- GPT-2 no usa un token `<UNK>` para palabras fuera del vocabulario; en su lugar, GPT-2 usa un tokenizador de codificación por pares de bytes (BPE), que descompone palabras en unidades de subpalabras, lo cual discutiremos en una sección posterior

- Usamos los tokens `<|endoftext|>` entre dos fuentes independientes de texto:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- Veamos qué sucede si tokenizamos el siguiente texto:

In [21]:
tokenizador = TokenizadorSimpleV1(vocabulario)

texto = "Hola, ¿te gusta el mate? ¿Es esto-- una prueba?"

# Esta celda producirá un error porque "Hola" no está en el vocabulario
# tokenizador.codificar(texto)

- Lo anterior produce un error porque la palabra "Hola" no está contenida en el vocabulario
- Para lidiar con tales casos, podemos agregar tokens especiales como `"<|unk|>"` al vocabulario para representar palabras desconocidas
- Ya que estamos extendiendo el vocabulario, agreguemos otro token llamado `"<|endoftext|>"` que se usa en el entrenamiento de GPT-2 para denotar el final de un texto (y también se usa entre textos concatenados, como si nuestro conjunto de datos de entrenamiento consiste en múltiples artículos, libros, etc.)

In [22]:
todos_tokens = sorted(list(set(preprocesado)))
todos_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocabulario = {token: entero for entero, token in enumerate(todos_tokens)}

In [23]:
len(vocabulario.items())

2106

In [24]:
for i, item in enumerate(list(vocabulario.items())[-5:]):
    print(item)

('“razonamiento', 2101)
('“todos', 2102)
('”', 2103)
('<|endoftext|>', 2104)
('<|unk|>', 2105)


- También necesitamos ajustar el tokenizador en consecuencia para que sepa cuándo y cómo usar el nuevo token `<unk>`

In [25]:
class TokenizadorSimpleV2:
    def __init__(self, vocabulario):
        self.str_a_int = vocabulario
        self.int_a_str = {i: s for s, i in vocabulario.items()}

    def codificar(self, texto):
        preprocesado = re.split(r'([,.:;?_!"()\']|--|\s)', texto)
        preprocesado = [item.strip() for item in preprocesado if item.strip()]
        preprocesado = [
            item if item in self.str_a_int
            else "<|unk|>" for item in preprocesado
        ]

        ids = [self.str_a_int[s] for s in preprocesado]
        return ids

    def decodificar(self, ids):
        texto = " ".join([self.int_a_str[i] for i in ids])
        # Reemplazamos espacios antes de las puntuaciones especificadas
        texto = re.sub(r'\s+([,.:;?!"()\'])', r'\1', texto)
        return texto

Intentemos tokenizar texto con el tokenizador modificado:

In [26]:
tokenizador = TokenizadorSimpleV2(vocabulario)

texto1 = "Hola, ¿te gusta el mate?"
texto2 = "En las terrazas iluminadas por el sol del palacio."

texto = " <|endoftext|> ".join((texto1, texto2))

print(texto)

Hola, ¿te gusta el mate? <|endoftext|> En las terrazas iluminadas por el sol del palacio.


In [27]:
tokenizador.codificar(texto)

[2105,
 2,
 2105,
 2105,
 849,
 2105,
 50,
 2104,
 125,
 1242,
 2105,
 2105,
 1596,
 849,
 1867,
 718,
 2105,
 23]

### Veamos la correspondencia entre IDs y texto:

In [28]:
ids_codificados = tokenizador.codificar(texto)
print("IDs:", ids_codificados)
print("\nTexto original:")
print(texto)
print("\nTexto decodificado:")
print(tokenizador.decodificar(ids_codificados))

IDs: [2105, 2, 2105, 2105, 849, 2105, 50, 2104, 125, 1242, 2105, 2105, 1596, 849, 1867, 718, 2105, 23]

Texto original:
Hola, ¿te gusta el mate? <|endoftext|> En las terrazas iluminadas por el sol del palacio.

Texto decodificado:
<|unk|>, <|unk|> <|unk|> el <|unk|>? <|endoftext|> En las <|unk|> <|unk|> por el sol del <|unk|>.


In [29]:
tokenizador.decodificar(tokenizador.codificar(texto))

'<|unk|>, <|unk|> <|unk|> el <|unk|>? <|endoftext|> En las <|unk|> <|unk|> por el sol del <|unk|>.'

## 2.5 Codificación por pares de bytes (BytePair encoding)

- GPT-2 usó codificación por pares de bytes (BPE) como su tokenizador
- Permite al modelo descomponer palabras que no están en su vocabulario predefinido en unidades de subpalabras más pequeñas o incluso caracteres individuales, permitiéndole manejar palabras fuera del vocabulario
- Por ejemplo, si el vocabulario de GPT-2 no tiene la palabra "palabradesconocida", podría tokenizarla como ["palabra", "desco", "nocida"] o alguna otra descomposición de subpalabras, dependiendo de sus fusiones BPE entrenadas
- El tokenizador BPE original se puede encontrar aquí: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- En este capítulo, usamos el tokenizador BPE de la biblioteca de código abierto [tiktoken](https://github.com/openai/tiktoken) de OpenAI, que implementa sus algoritmos principales en Rust para mejorar el rendimiento computacional

In [30]:
import importlib
import tiktoken

print("Versión de tiktoken:", importlib.metadata.version("tiktoken"))

Versión de tiktoken: 0.12.0


In [31]:
tokenizador = tiktoken.get_encoding("gpt2")

In [32]:
texto = (
    "Hola, ¿te gusta el mate? <|endoftext|> En las terrazas iluminadas"
     " por el sol de algunLugarDesconocido."
)

enteros = tokenizador.encode(texto, allowed_special={"<|endoftext|>"})

print(enteros)

[39, 5708, 11, 1587, 123, 660, 35253, 64, 1288, 16133, 30, 220, 50256, 2039, 39990, 1059, 3247, 292, 4229, 7230, 38768, 16964, 1288, 1540, 390, 435, 7145, 43, 35652, 5960, 1102, 420, 17305, 13]


### Veamos la correspondencia token por token:

In [33]:
print("IDs:", enteros)
print("\nCorrespondencia token por token:")
for i, id_token in enumerate(enteros):
    print(f"  {id_token:5d} --> {tokenizador.decode([id_token])!r}")

IDs: [39, 5708, 11, 1587, 123, 660, 35253, 64, 1288, 16133, 30, 220, 50256, 2039, 39990, 1059, 3247, 292, 4229, 7230, 38768, 16964, 1288, 1540, 390, 435, 7145, 43, 35652, 5960, 1102, 420, 17305, 13]

Correspondencia token por token:
     39 --> 'H'
   5708 --> 'ola'
     11 --> ','
   1587 --> ' �'
    123 --> '�'
    660 --> 'te'
  35253 --> ' gust'
     64 --> 'a'
   1288 --> ' el'
  16133 --> ' mate'
     30 --> '?'
    220 --> ' '
  50256 --> '<|endoftext|>'
   2039 --> ' En'
  39990 --> ' las'
   1059 --> ' ter'
   3247 --> 'raz'
    292 --> 'as'
   4229 --> ' il'
   7230 --> 'umin'
  38768 --> 'adas'
  16964 --> ' por'
   1288 --> ' el'
   1540 --> ' sol'
    390 --> ' de'
    435 --> ' al'
   7145 --> 'gun'
     43 --> 'L'
  35652 --> 'ugar'
   5960 --> 'Des'
   1102 --> 'con'
    420 --> 'oc'
  17305 --> 'ido'
     13 --> '.'


In [34]:
cadenas = tokenizador.decode(enteros)

print(cadenas)

Hola, ¿te gusta el mate? <|endoftext|> En las terrazas iluminadas por el sol de algunLugarDesconocido.


- Los tokenizadores BPE descomponen palabras desconocidas en subpalabras y caracteres individuales:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 Muestreo de datos con una ventana deslizante

- Entrenamos LLMs para generar una palabra a la vez, así que queremos preparar los datos de entrenamiento en consecuencia donde la siguiente palabra en una secuencia representa el objetivo a predecir:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [35]:
with open(ruta_archivo_drive, "r", encoding="utf-8") as archivo:
    texto_crudo = archivo.read()

texto_codificado = tokenizador.encode(texto_crudo)
print(len(texto_codificado))

11530


- Para cada fragmento de texto, queremos las entradas y los objetivos
- Como queremos que el modelo prediga la siguiente palabra, los objetivos son las entradas desplazadas una posición a la derecha

In [36]:
muestra_codificada = texto_codificado[50:]

In [38]:
tamaño_contexto = 4

x = muestra_codificada[:tamaño_contexto]
y = muestra_codificada[1:tamaño_contexto+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [28533, 64, 390, 8591]
y:      [64, 390, 8591, 2386]


### Veamos qué texto representan:

In [39]:
print("Entrada (x):")
print("  IDs:", x)
print("  Texto:", tokenizador.decode(x))
print("\nObjetivo (y):")
print("  IDs:", y)
print("  Texto:", tokenizador.decode(y))

Entrada (x):
  IDs: [28533, 64, 390, 8591]
  Texto:  quinta de la

Objetivo (y):
  IDs: [64, 390, 8591, 2386]
  Texto: a de la cal


- Una por una, la predicción se vería de la siguiente manera:

In [40]:
for i in range(1, tamaño_contexto+1):
    contexto = muestra_codificada[:i]
    deseado = muestra_codificada[i]

    print(contexto, "---->", deseado)

[28533] ----> 64
[28533, 64] ----> 390
[28533, 64, 390] ----> 8591
[28533, 64, 390, 8591] ----> 2386


In [41]:
for i in range(1, tamaño_contexto+1):
    contexto = muestra_codificada[:i]
    deseado = muestra_codificada[i]

    print(tokenizador.decode(contexto), "---->", tokenizador.decode([deseado]))

 quint ----> a
 quinta ---->  de
 quinta de ---->  la
 quinta de la ---->  cal


- Nos encargaremos de la predicción de la siguiente palabra en un capítulo posterior después de cubrir el mecanismo de atención
- Por ahora, implementamos un simple data loader que itera sobre el dataset de entrada y devuelve las entradas y los objetivos desplazados por uno

- Instalamos e importamos PyTorch (ver Apéndice A para consejos de instalación)

In [42]:
import torch
print("Versión de PyTorch:", torch.__version__)

Versión de PyTorch: 2.8.0+cu126


- Usamos un enfoque de ventana deslizante, cambiando la posición en +1:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- Creamos dataset y dataloader que extraen fragmentos del dataset de texto de entrada

In [43]:
from torch.utils.data import Dataset, DataLoader


class DatasetGPTV1(Dataset):
    def __init__(self, txt, tokenizador, longitud_maxima, paso):
        self.ids_entrada = []
        self.ids_objetivo = []

        # Tokenizamos el texto completo
        ids_tokens = tokenizador.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(ids_tokens) > longitud_maxima, "El número de entradas tokenizadas debe ser al menos igual a longitud_maxima+1"

        # Usamos una ventana deslizante para fragmentar el libro en secuencias superpuestas de longitud_maxima
        for i in range(0, len(ids_tokens) - longitud_maxima, paso):
            fragmento_entrada = ids_tokens[i:i + longitud_maxima]
            fragmento_objetivo = ids_tokens[i + 1: i + longitud_maxima + 1]
            self.ids_entrada.append(torch.tensor(fragmento_entrada))
            self.ids_objetivo.append(torch.tensor(fragmento_objetivo))

    def __len__(self):
        return len(self.ids_entrada)

    def __getitem__(self, idx):
        return self.ids_entrada[idx], self.ids_objetivo[idx]

In [44]:
def crear_dataloader_v1(txt, tamaño_batch=4, longitud_maxima=256,
                         paso=128, mezclar=True, descartar_ultimo=True,
                         num_trabajadores=0):

    # Inicializamos el tokenizador
    tokenizador = tiktoken.get_encoding("gpt2")

    # Creamos el dataset
    dataset = DatasetGPTV1(txt, tokenizador, longitud_maxima, paso)

    # Creamos el dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=tamaño_batch,
        shuffle=mezclar,
        drop_last=descartar_ultimo,
        num_workers=num_trabajadores
    )

    return dataloader

- Probemos el dataloader con un tamaño de batch de 1 para un LLM con un tamaño de contexto de 4:

In [45]:
with open(ruta_archivo_drive, "r", encoding="utf-8") as archivo:
    texto_crudo = archivo.read()

In [46]:
dataloader = crear_dataloader_v1(
    texto_crudo, tamaño_batch=1, longitud_maxima=4, paso=1, mezclar=False
)

iterador_datos = iter(dataloader)
primer_batch = next(iterador_datos)
print(primer_batch)

[tensor([[16587,    78,   257,  8591]]), tensor([[   78,   257,  8591, 11644]])]


### Veamos qué texto representa este batch:

In [47]:
print("Entrada:", primer_batch[0])
print("Texto de entrada:", tokenizador.decode(primer_batch[0][0].tolist()))
print("\nObjetivo:", primer_batch[1])
print("Texto objetivo:", tokenizador.decode(primer_batch[1][0].tolist()))

Entrada: tensor([[16587,    78,   257,  8591]])
Texto de entrada: Debo a la

Objetivo: tensor([[   78,   257,  8591, 11644]])
Texto objetivo: o a la conj


In [50]:
segundo_batch = next(iterador_datos)
print(segundo_batch)

[tensor([[ 2574,  1658,   431,  7639],
        [ 7949,  1155, 15498,  1288],
        [16245,    78,   390,   555],
        [ 1162,   445,   273,   551],
        [  555,    64, 28533,    64],
        [  390,  8591,  2386,   293],
        [12822,  4450,    11,   551],
        [36692,  2185,    73, 29690]]), tensor([[ 1658,   431,  7639,  7949],
        [ 1155, 15498,  1288, 16245],
        [   78,   390,   555,  1162],
        [  445,   273,   551,   555],
        [   64, 28533,    64,   390],
        [ 8591,  2386,   293, 12822],
        [ 4450,    11,   551, 36692],
        [ 2185,    73, 29690,    26]])]


### Veamos el segundo batch:

In [51]:
print("Entrada:", segundo_batch[0])
print("Texto de entrada:", tokenizador.decode(segundo_batch[0][0].tolist()))
print("\nObjetivo:", segundo_batch[1])
print("Texto objetivo:", tokenizador.decode(segundo_batch[1][0].tolist()))

Entrada: tensor([[ 2574,  1658,   431,  7639],
        [ 7949,  1155, 15498,  1288],
        [16245,    78,   390,   555],
        [ 1162,   445,   273,   551],
        [  555,    64, 28533,    64],
        [  390,  8591,  2386,   293],
        [12822,  4450,    11,   551],
        [36692,  2185,    73, 29690]])
Texto de entrada:  El espejo

Objetivo: tensor([[ 1658,   431,  7639,  7949],
        [ 1155, 15498,  1288, 16245],
        [   78,   390,   555,  1162],
        [  445,   273,   551,   555],
        [   64, 28533,    64,   390],
        [ 8591,  2386,   293, 12822],
        [ 4450,    11,   551, 36692],
        [ 2185,    73, 29690,    26]])
Texto objetivo:  espejo inqu


- Un ejemplo usando paso igual a la longitud del contexto (aquí: 4) como se muestra a continuación:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- También podemos crear salidas en batch
- Notemos que aumentamos el paso aquí para que no tengamos superposiciones entre los batches, ya que más superposición podría llevar a un aumento del sobreajuste

In [52]:
dataloader = crear_dataloader_v1(texto_crudo, tamaño_batch=8, longitud_maxima=4, paso=4, mezclar=False)

iterador_datos = iter(dataloader)
entradas, objetivos = next(iterador_datos)
print("Entradas:\n", entradas)
print("\nObjetivos:\n", objetivos)

Entradas:
 tensor([[16587,    78,   257,  8591],
        [11644, 49652, 18840,   390],
        [  555,  1658,   431,  7639],
        [  331,   390,   555,    64],
        [ 2207,   291,    75,   404],
        [ 5507,  1288,  1715,   549],
        [ 3036,  1153,    78,   390],
        [  471,    80,  5657,    13]])

Objetivos:
 tensor([[   78,   257,  8591, 11644],
        [49652, 18840,   390,   555],
        [ 1658,   431,  7639,   331],
        [  390,   555,    64,  2207],
        [  291,    75,   404,  5507],
        [ 1288,  1715,   549,  3036],
        [ 1153,    78,   390,   471],
        [   80,  5657,    13,  2574]])


### Veamos qué texto representan estos batches (primeras 3 muestras):

In [53]:
print("Mostrando las primeras 3 muestras del batch:\n")
for i in range(min(3, len(entradas))):
    print(f"Muestra {i+1}:")
    print(f"  Entrada (IDs): {entradas[i].tolist()}")
    print(f"  Entrada (texto): {tokenizador.decode(entradas[i].tolist())!r}")
    print(f"  Objetivo (IDs): {objetivos[i].tolist()}")
    print(f"  Objetivo (texto): {tokenizador.decode(objetivos[i].tolist())!r}")
    print()

Mostrando las primeras 3 muestras del batch:

Muestra 1:
  Entrada (IDs): [16587, 78, 257, 8591]
  Entrada (texto): 'Debo a la'
  Objetivo (IDs): [78, 257, 8591, 11644]
  Objetivo (texto): 'o a la conj'

Muestra 2:
  Entrada (IDs): [11644, 49652, 18840, 390]
  Entrada (texto): ' conjunción de'
  Objetivo (IDs): [49652, 18840, 390, 555]
  Objetivo (texto): 'unción de un'

Muestra 3:
  Entrada (IDs): [555, 1658, 431, 7639]
  Entrada (texto): ' un espejo'
  Objetivo (IDs): [1658, 431, 7639, 331]
  Objetivo (texto): ' espejo y'



## 2.7 Creando embeddings de tokens

- Los datos ya están casi listos para un LLM
- Pero por último embedamos los tokens en una representación vectorial continua usando una capa de embedding
- Usualmente, estas capas de embedding son parte del LLM mismo y se actualizan (entrenan) durante el entrenamiento del modelo

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- Supongamos que tenemos los siguientes cuatro ejemplos de entrada con IDs de entrada 2, 3, 5 y 1 (después de la tokenización):

In [54]:
ids_entrada = torch.tensor([2, 3, 5, 1])

- Por simplicidad, supongamos que tenemos un vocabulario pequeño de solo 6 palabras y queremos crear embeddings de tamaño 3:

In [55]:
tamaño_vocab = 6
dim_salida = 3

torch.manual_seed(123)
capa_embedding = torch.nn.Embedding(tamaño_vocab, dim_salida)

- Esto resultaría en una matriz de pesos de 6x3:

In [56]:
print(capa_embedding.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- Para aquellos que están familiarizados con la codificación one-hot, el enfoque de la capa de embedding anterior es esencialmente solo una forma más eficiente de implementar la codificación one-hot seguida de la multiplicación de matrices en una capa completamente conectada
- Porque la capa de embedding es solo una implementación más eficiente que es equivalente al enfoque de codificación one-hot y multiplicación de matrices, puede verse como una capa de red neuronal que puede ser optimizada mediante backpropagation

- Para convertir un token con id 3 en un vector de 3 dimensiones, hacemos lo siguiente:

In [57]:
print(capa_embedding(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- Notemos que lo anterior es la cuarta fila en la matriz de pesos `capa_embedding`
- Para embeber todos los cuatro valores de `ids_entrada` anteriores, hacemos

In [58]:
print(capa_embedding(ids_entrada))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- Una capa de embedding es esencialmente una operación de búsqueda:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

## 2.8 Codificando posiciones de palabras

- La capa de embedding convierte IDs en representaciones vectoriales idénticas independientemente de dónde se encuentren en la secuencia de entrada:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- Los embeddings posicionales se combinan con el vector de embedding de tokens para formar los embeddings de entrada para un modelo de lenguaje grande:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- El codificador BytePair tiene un tamaño de vocabulario de 50,257
- Supongamos que queremos codificar los tokens de entrada en una representación vectorial de 256 dimensiones:

In [66]:
tamaño_vocab = 50257
dim_salida = 256

capa_embedding_tokens = torch.nn.Embedding(tamaño_vocab, dim_salida)

- Si tomamos muestras de datos del dataloader, embedamos los tokens en cada batch en un vector de 256 dimensiones
- Si tenemos un tamaño de batch de 8 con 4 tokens cada uno, esto resulta en un tensor de 8 x 4 x 256:

In [67]:
longitud_maxima = 4
dataloader = crear_dataloader_v1(
    texto_crudo, tamaño_batch=8, longitud_maxima=longitud_maxima,
    paso=longitud_maxima, mezclar=False
)
iterador_datos = iter(dataloader)
entradas, objetivos = next(iterador_datos)

In [68]:
print("IDs de tokens:\n", entradas)
print("\nForma de las entradas:\n", entradas.shape)

IDs de tokens:
 tensor([[16587,    78,   257,  8591],
        [11644, 49652, 18840,   390],
        [  555,  1658,   431,  7639],
        [  331,   390,   555,    64],
        [ 2207,   291,    75,   404],
        [ 5507,  1288,  1715,   549],
        [ 3036,  1153,    78,   390],
        [  471,    80,  5657,    13]])

Forma de las entradas:
 torch.Size([8, 4])


### Veamos qué palabras representan estos IDs (primeras 3 muestras):

In [69]:
print("Mostrando las primeras 3 muestras del batch:\n")
for i in range(min(3, len(entradas))):
    print(f"Muestra {i+1}:")
    print(f"  IDs: {entradas[i].tolist()}")
    print(f"  Texto: {tokenizador.decode(entradas[i].tolist())!r}")
    print()

Mostrando las primeras 3 muestras del batch:

Muestra 1:
  IDs: [16587, 78, 257, 8591]
  Texto: 'Debo a la'

Muestra 2:
  IDs: [11644, 49652, 18840, 390]
  Texto: ' conjunción de'

Muestra 3:
  IDs: [555, 1658, 431, 7639]
  Texto: ' un espejo'



In [70]:
embeddings_tokens = capa_embedding_tokens(entradas)
print(embeddings_tokens.shape)

# descomentá y ejecutá la siguiente línea para ver cómo se ven los embeddings
# print(embeddings_tokens)

torch.Size([8, 4, 256])


- GPT-2 usa embeddings de posición absolutos, así que simplemente creamos otra capa de embedding:

In [71]:
longitud_contexto = longitud_maxima
capa_embedding_posicion = torch.nn.Embedding(longitud_contexto, dim_salida)

# descomentá y ejecutá la siguiente línea para ver cómo se ven los pesos de la capa de embedding
# print(capa_embedding_posicion.weight)

In [72]:
embeddings_posicion = capa_embedding_posicion(torch.arange(longitud_maxima))
print(embeddings_posicion.shape)

# descomentá y ejecutá la siguiente línea para ver cómo se ven los embeddings
# print(embeddings_posicion)

torch.Size([4, 256])


- Para crear los embeddings de entrada usados en un LLM, simplemente sumamos los embeddings de tokens y los embeddings posicionales:

In [73]:
embeddings_entrada = embeddings_tokens + embeddings_posicion
print(embeddings_entrada.shape)

# descomentá y ejecutá la siguiente línea para ver cómo se ven los embeddings
# print(embeddings_entrada)

torch.Size([8, 4, 256])


- En la fase inicial del flujo de procesamiento de entrada, el texto de entrada se segmenta en tokens separados
- Después de esta segmentación, estos tokens se transforman en IDs de tokens basados en un vocabulario predefinido:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# Resumen y puntos clave

Este cuaderno ilustra el proceso completo de preparación de datos de texto para entrenar un LLM:

1. **Tokenización**: Dividimos el texto en unidades más pequeñas (tokens)
2. **Vocabulario**: Construimos un diccionario que mapea tokens a números
3. **Tokens especiales**: Agregamos tokens como `<|endoftext|>` y `<|unk|>` para manejar casos especiales
4. **BytePair Encoding (BPE)**: Un método más sofisticado que maneja palabras desconocidas descomponiéndolas en subpalabras
5. **Data Loader**: Creamos batches de datos con una ventana deslizante para entrenar el modelo
6. **Embeddings**: Convertimos los tokens en vectores densos que el modelo puede procesar
7. **Embeddings posicionales**: Agregamos información sobre la posición de cada token en la secuencia

En este caso particular, trabajamos con el texto "Tlön, Uqbar, Orbis Tertius" de Jorge Luis Borges, demostrando cómo estos conceptos se aplican a textos en español.